In [ ]:
!nvidia-smi


Wed May 18 09:15:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget https://svn.spraakdata.gu.se/sb-arkiv/pub/dalaj/datasetDaLAJsplit.csv


--2022-05-18 09:15:17--  https://svn.spraakdata.gu.se/sb-arkiv/pub/dalaj/datasetDaLAJsplit.csv
Resolving svn.spraakdata.gu.se (svn.spraakdata.gu.se)... 130.241.135.158
Connecting to svn.spraakdata.gu.se (svn.spraakdata.gu.se)|130.241.135.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1534736 (1.5M) [text/csv]
Saving to: ‘datasetDaLAJsplit.csv’

datasetDaLAJsplit.c 100%[===================>]   1.46M  8.41MB/s    in 0.2s    

2022-05-18 09:15:18 (8.41 MB/s) - ‘datasetDaLAJsplit.csv’ saved [1534736/1534736]



In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics



In [ ]:
dalaj_df = pd.read_csv("/content/datasetDaLAJsplit.csv")

In [ ]:
#@title Fix arrangement Train

train_df = dalaj_df[dalaj_df['split'] == "train" ]

sorted_items = []
for idx, item in train_df.iterrows():
  correct_dict = {"Text": item["corrected sentence"],"Label":1}
  sorted_items.append(correct_dict)

  incorrect_dict = {"Text": item["original sentence"],"Label":0}
  sorted_items.append(incorrect_dict)

final_train_df = pd.DataFrame(sorted_items)
from sklearn.utils import shuffle
final_train_df = shuffle(final_train_df)

In [ ]:
final_train_df

,Text,Label
759,Trots alla argument om finlandssvenskan och fi...,0
2367,Jag tycker om den plasten efter jag blir lyckl...,0
2532,Så hon kommer att bli jätteglad att vi hälsar ...,1
2022,Alla har egna åsikter och behöver avstånd .,1
5177,En del anser att världens länder blir kolonier...,0
...,...,...
241,Enligt historian så var Finland en del av Sver...,0
678,Jag lagar mat åt min man i köket .,1
6759,Det var där för jag skaffade henne .,0
5005,Vad kan vara viktigare av biblioteket här i om...,0


In [ ]:
#@title Fix arrangement for test

test_df = dalaj_df[dalaj_df['split'] == "test" ]

sorted_items = []
for idx, item in test_df.iterrows():
  correct_dict = {"Text": item["corrected sentence"],"Label":1}
  sorted_items.append(correct_dict)

  incorrect_dict = {"Text": item["original sentence"],"Label":0}
  sorted_items.append(incorrect_dict)

final_test_df = pd.DataFrame(sorted_items)
from sklearn.utils import shuffle
final_test_df = shuffle(final_test_df)


In [ ]:
final_test_df

,Text,Label
286,Jag hoppas kunna lära mig snabbt och börja sök...,1
31,Från utseendet kan vi inte förstå om man är pr...,0
502,Familjen måste alltid delta i allting i livet ...,1
357,Den största fördelen med platsen är att jag ka...,0
351,Vidare kan jag tar de här aktiviteterna som er...,0
...,...,...
458,Då uppstår frågan om jag ska ta ett extra pass...,1
127,Jag tror att författaren har skrivit den här b...,0
364,Det är nödvändigt att ha en plats som kan påve...,1
23,För mig är kläder viktigt men inte så mycket .,0


In [ ]:
#@title sentence model
#!pip install -U sentence-transformers
#from sentence_transformers import SentenceTransformer, models

#model_dir = 'Peltarion/xlm-roberta-longformer-base-4096'
#mod = "KB/bert-base-swedish-cased"
#sen_xlmr = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

#word_embedding_model = models.Transformer(mod, tokenizer_name_or_path= mod , max_seq_length=512)
#pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

#model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

#EDA

In [ ]:
#@title EDA
!git clone https://github.com/mosh98/swe_aug.git
!pip install -r /content/swe_aug/reqs.txt
!wget https://www.ida.liu.se/divisions/hcs/nlplab/swectors/swectors-300dim.txt.bz2
!bzip2 -dk /content/swectors-300dim.txt.bz2
word_vec_path = '/content/swectors-300dim.txt'
from swe_aug import EDA
aug = EDA.Enkel_Data_Augmentation(word_vec_path)

# Fraction of Dataset


In [ ]:
!pip install transformers

In [ ]:
!pip install MultiEncoder==0.0.6

In [ ]:
mod = "KB/bert-base-swedish-cased"
from mle import multi_layer_encoder
le = multi_layer_encoder.multi_layer_encoder(mod)

In [ ]:
#@title

from sklearn.model_selection import train_test_split


def frac(dataframe, fraction,label):
    """Returns fraction of data"""
    if fraction == 1.0:
      return dataframe
      
    y = dataframe[label]
    train, test = train_test_split(dataframe,train_size=fraction,stratify = y)
    
    return train

def encode_df(dataframe, mod=None,col = "text"):
  encoded = []
  for idx, item in dataframe.iterrows():
        list_of_encoded_inputs, dect = le.multi_encode(item.Text)
        encoded.append(list_of_encoded_inputs[1])
        #encoded.append(model.encode(item[col])) 

  return encoded


reports = [] #clf_report (Before Augment, After Augment)


In [ ]:
#@title
test_embed = []

y_test = final_test_df.Label

for idx, item in final_test_df.iterrows():
      list_of_encoded_inputs, dect = le.multi_encode(item.Text)
      test_embed.append(list_of_encoded_inputs[1])
#test_embed.append(model.encode(item.Text))
  

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
#@title
#from swe_aug.Other_Techniques import Text_Cropping
#frag = Text_Cropping.cropper(percent = 0.25)
!pip install SpaceAugmentation
from aug import Augmentation

ag = Augmentation.Augmentation()

In [ ]:
#@title
from swe_aug.Other_Techniques import Type_SR
aug = Type_SR.type_DA(word_vec_path)

Already downloaded a model for the 'sv' language


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#@title Training Loop
from sklearn.linear_model import SGDClassifier

split_percentage = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

label = "Label"

for percentage in split_percentage:
    print("______________________________________________")
    
    print("Percentage",percentage)
    
    temp_train_df = frac(final_train_df, percentage/100,label)
    y_train = temp_train_df.Label
    
    print("Before Augmentation size: ",temp_train_df.shape)
    
    train_embed = encode_df( temp_train_df,  col="Text")

    logreg = SGDClassifier(max_iter=3000,n_jobs=-1)
    logreg.fit(train_embed, y_train)
    
    y_pred = logreg.predict(test_embed)
    
    r_1 = classification_report(y_test, y_pred)
    print("Before Augmentation")
    print(r_1)
    print(" ")

    # picking out bad samples
    incorrect_df = temp_train_df[temp_train_df['Label'] == 0]

    #Augmentation

    aug_samples = []
    for idx, item in incorrect_df.iterrows():
      txt = item.Text
      lab = item.Label
      list_of_augs = aug.type_synonym_sr(txt, token_type = "NOUN", n = 2)
      
      for element in list_of_augs:
          aug_samples.append({"Text":' '.join(element),"Label":lab})

    
    new_aug_samples = pd.DataFrame(aug_samples)
    

    
    
    
    new_df = pd.concat([temp_train_df,new_aug_samples],ignore_index=True)
    
    print("After Augmentation size: ",new_df.shape[0])


    #augmented train test

    #encode
    train_embed = encode_df( new_df,  col="Text")
    y_train = new_df.Label
    
    logreg_ = SGDClassifier(max_iter=3000, n_jobs=-1)
    logreg_.fit(train_embed, y_train)

    y_pred = logreg_.predict(test_embed)


    r_2 = classification_report(y_test, y_pred)
    print("After Augmentation")
    print(r_2)

    #save the reports
    reports.append((percentage, r_1,r_2))



    print("______________________________________________")








______________________________________________
Percentage 10
Before Augmentation size:  (768, 2)
Before Augmentation
              precision    recall  f1-score   support

           0       0.55      0.72      0.62       444
           1       0.59      0.40      0.48       444

    accuracy                           0.56       888
   macro avg       0.57      0.56      0.55       888
weighted avg       0.57      0.56      0.55       888

 
After Augmentation size:  1536
After Augmentation
              precision    recall  f1-score   support

           0       0.54      0.85      0.66       444
           1       0.64      0.26      0.37       444

    accuracy                           0.56       888
   macro avg       0.59      0.56      0.51       888
weighted avg       0.59      0.56      0.51       888

______________________________________________
______________________________________________
Percentage 20
Before Augmentation size:  (1536, 2)
Before Augmentation
            

In [ ]:

!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models

#model_dir = 'Peltarion/xlm-roberta-longformer-base-4096'
#mod = "KB/bert-base-swedish-cased"
sen_xlmr = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

word_embedding_model = models.Transformer(sen_xlmr, tokenizer_name_or_path= sen_xlmr , max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
    # picking out bad samples
incorrect_df = final_train_df[final_train_df['Label'] == 0]

    #Augmentation

aug_samples = []
original_augmented = [] #every element : orginal_text, (Tuple of augmented sentences)
for idx, item in incorrect_df.iterrows():
      txt = item.Text
      lab = item.Label
      list_of_augs = aug.type_synonym_sr(txt, token_type = "NOUN", n = 2)
      
      temp_list = []
      for element in list_of_augs:
            aug_samples.append({"Text":' '.join(element),"Label":lab})
            temp_list.append(' '.join(element))
      
      original_augmented.append((txt,temp_list))

    

new_aug_samples = pd.DataFrame(aug_samples)

In [ ]:
from sentence_transformers import SentenceTransformer, util


def checkSimilarity(reference_string, list_of_strings, model):
    """
    This function takes a reference string and a list of strings and returns a list of strings that are similar to the reference string.

    :param reference_string:
    :param list_of_strings:
    :param model: sentence transformer model
    :return:
    """
    #1. Encode using Sentence Transformer
    #2. Calculate cosine similarity
    #3. Append the similirty value to the list
    # caclulcate percentage of elements below 0.9
    # return the list, and the percentage of elements below 0.9

    reference_encoded = model.encode(reference_string,convert_to_numpy=True) #encoded reference string
    list_of_cosine_similarity = []
    for string in list_of_strings:
        #print(string)
        encoded_string = model.encode(string,convert_to_numpy=True)

        #find coside similairity of enocded String and reference string
        similarity = util.cos_sim(reference_encoded, encoded_string)
        list_of_cosine_similarity.append(similarity.item())
    #print(list_of_cosine_similarity)
    #sum(i > 5 for i in j)
    percentage_of_elements_below_0_9 = sum(i < 0.9 for i in list_of_cosine_similarity) / len(list_of_cosine_similarity)

    return sum(i < 0.95 for i in list_of_cosine_similarity),percentage_of_elements_below_0_9

In [ ]:
low_semantic_sentences = []
for item in original_augmented:
  num, percentage = checkSimilarity(item[0],item[1],model)
  low_semantic_sentences.append(num)

In [ ]:
print("Original Sentences:",len(original_augmented))
print("Num of Augmented Sentences:",len(original_augmented)*2)


Original Sentences: 3841
Num of Augmented Sentences: 7682


In [ ]:
print("Num of Bad Augmented Sentences:",sum(low_semantic_sentences))


Num of Bad Augmented Sentences: 2782


In [ ]:
num_of_aug_samples = len(original_augmented)*2
print("Percentage of Bad Augmented Sentences:",(sum(low_semantic_sentences)/num_of_aug_samples)*100)

Percentage of Bad Augmented Sentences: 36.21452746680552
